---
# Chapter 1: Setup


## Step 1: Install Libraries and checking the dataset

In [58]:
#install the nessecary Library
!pip install reportlab pandas pillow

In [59]:
from reportlab.pdfgen import canvas #to make the pdf
from reportlab.lib.pagesizes import letter #to make the pdf in letter size
from reportlab.lib.utils import ImageReader #to include our image
from PIL import Image #to auto resize our image
import pandas as pd #to check/make table
import os #to interact with the machine memory
from io import BytesIO #handle image in the machine memory

## Step 2: Upload the Sample CSV

1. Have your 'Garment Bag Tag_Dataset.csv' in your machine
2. In Colab, click the folder icon on the left
3. Click the upload icon and navigate to the 'Garment Bag Tag_Dataset.csv' file

## Step 3: Checking the dataset

In [60]:
#read the CSV file
df = pd.read_csv('Garment_Bag_Tag_Dataset.csv')
print(f"Making labels for {len(df)} items...")

Making labels for 29 items...


In [61]:
#checking the first table
print(f"Loaded {len(df)} garments")
print(f"Will create {(len(df)+3)//4} pages")
print("\nFirst garment:")
print(df.iloc[0])

Loaded 29 garments
Will create 8 pages

First garment:
identifier                                              C2025-0169-LL
designer                                                Mary McFadden
description         Purple charmeuse long sleeve evening gown with...
item_type                                               Evening Dress
collection                                Spring American Indian 1989
storage_location                                             Rack A-1
condition                                                        Fair
measurements                                                L:49 B:34
Image_URL                                           C2025-0169-LL.jpg
Name: 0, dtype: object


## Step 4: Creating images folder to upload the images.

1. We need to make a 'images' folder
2. click the three-dots on the right side of the folder
3. click the 'upload'
4. navigate to our image set

In [62]:
#create images folder if it doesn't exist
if not os.path.exists('images'):
    os.makedirs('images')
    print("Created 'images' folder")
else:
    print("'images' folder already exists")

'images' folder already exists


## Step 5: checking the images files

In [63]:
#checking the qty. of the images
csv_images = df['Image_URL'].dropna().tolist()
print(f"find {len(csv_images)} images")

find 29 images


In [64]:
#check the image files
if os.path.exists('images'):
    folder_images = os.listdir('images')
    print(f"Found {len(folder_images)} images in folder")

    # Check which ones match
    missing = []
    found = []

    for img in csv_images:
        if img in folder_images:
            found.append(img)
        else:
            missing.append(img)

    print(f"\nMatched: {len(found)}")
    print(f"Missing: {len(missing)}")

    if missing:
        print("\nMissing images:")
        for m in missing[:5]:  # Show first 5 missing
            print(f"  - {m}")
else:
    print("No images folder found")

Found 27 images in folder

Matched: 27
Missing: 2

Missing images:
  - C2025-0187-LL.jpg
  - C2025-0188-LL.jpg


In [65]:
#image location check
test_batch = df.head(5)
for i in range(5):
    item = test_batch.iloc[i]
    filename = str(item['Image_URL'])
    image_path = f'images/{filename}'
    print(f"Item {i+1}: {item['identifier']}")
    print(f"  Looking for: {image_path}")
    print(f"  File exists: {os.path.exists(image_path)}")

Item 1: C2025-0169-LL
  Looking for: images/C2025-0169-LL.jpg
  File exists: True
Item 2: C2025-0170-LL
  Looking for: images/C2025-0170-LL.jpg
  File exists: True
Item 3: C2025-0171-LL
  Looking for: images/C2025-0171-LL.jpg
  File exists: True
Item 4: C2025-0172-LL
  Looking for: images/C2025-0172-LL.jpg
  File exists: True
Item 5: C2025-0173-LL
  Looking for: images/C2025-0173-LL.jpg
  File exists: True


---

# Chapter 2: Test the first page of our Garment label

## Step 1: setting the design and layout

In [66]:
#Create Test One Page PDF
pdf = canvas.Canvas("TEST_labels.pdf", pagesize=letter)
width, height = letter  # 8.5 x 11 inches

label_width = width / 2    # Half page width
label_height = height / 2   # Half page height

# Get first 4 items only
test_batch = df.head(4)

positions = [
    (0, label_height),           # Top-left
    (label_width, label_height), # Top-right
    (0, 0),                      # Bottom-left
    (label_width, 0)             # Bottom-right
]

# ----------------CUSTOMIZE YOUR LABEL AFTER THIS----------------
# Draw 4 test labels
for i in range(4):
    if i < len(test_batch):
        item = test_batch.iloc[i]
        x, y = positions[i]

        # Draw border
        pdf.rect(x, y, label_width, label_height)

        # Center position for text
        center_x = x + (label_width / 2)

        # Add the table columns
        pdf.setFont("Helvetica-Bold", 20)
        pdf.drawCentredString(center_x, y + 350, str(item['identifier']))
        pdf.setFont("Helvetica", 10)
        pdf.drawCentredString(center_x, y + 330, str(item['designer']))

        # Add image if exists
        if pd.notna(item.get('Image_URL', '')):
            image_path = f"images/{item['Image_URL']}"
            if os.path.exists(image_path):
                try:
                    img = Image.open(image_path)
                    img.thumbnail((200, 200), Image.Resampling.LANCZOS)
                    img_buffer = BytesIO()
                    img.save(img_buffer, format='JPEG', quality=100)
                    img_buffer.seek(0)
                    img_reader = ImageReader(img_buffer)
                    pdf.drawImage(img_reader, center_x - 75, y + 120,
                                width=150, height=180, preserveAspectRatio=True)
                except:
                    pdf.drawCentredString(center_x, y + 200, "[Image Error]")
            else:
                pdf.drawCentredString(center_x, y + 200, "[No Image]")

        pdf.setFont("Helvetica-Bold", 11)
        pdf.drawCentredString(center_x, y + 60, str(item['item_type']))

        # Add institution /please update this with yours
        pdf.setFont("Helvetica-Bold", 9)
        pdf.drawCentredString(center_x, y + 30, "Pratt Fashion Design")
        pdf.drawCentredString(center_x, y + 15, "STUDY COLLECTION")

##Step 2: Testing the long description

We have variations in the length of descriptions. To address this, we need to split longer descriptions into two lines or display '...' if they exceed that length. The idea is to trim by word rather than by character count. This is a good opportunity to use the [textwarp](https://docs.python.org/3/library/textwrap.html) library.

In [67]:
#testing your descriptions before PDF
import textwrap

test_descriptions = [
    "Short description",
    "This is a medium length description that needs to break into two lines if possible",
    "This is a very long description that will definitely need to be cut off with ellipsis because it contains way too much text to fit in two lines on our label"
]

for desc in test_descriptions:
    print(f"Original ({len(desc)} chars): {desc}")

    # Use textwrap to split into lines (max 70 chars, max 2 lines)
    lines = textwrap.wrap(desc, width=70, max_lines=2, placeholder="...")

    # Print the result
    if len(lines) >= 1:
        print(f"  Line 1: {lines[0]}")
    if len(lines) >= 2:
        print(f"  Line 2: {lines[1]}")

    print()

Original (17 chars): Short description
  Line 1: Short description

Original (82 chars): This is a medium length description that needs to break into two lines if possible
  Line 1: This is a medium length description that needs to break into two lines
  Line 2: if possible

Original (156 chars): This is a very long description that will definitely need to be cut off with ellipsis because it contains way too much text to fit in two lines on our label
  Line 1: This is a very long description that will definitely need to be cut
  Line 2: off with ellipsis because it contains way too much text to fit in...



## Step 3: Testing our first page with 4 tags

In [68]:
#create TEST PDF
pdf = canvas.Canvas("TEST_labels_with_desc.pdf", pagesize=letter)
width, height = letter
label_width = width / 2
label_height = height / 2

#get first 4 items
test_batch = df.head(4)

positions = [
    (0, label_height),
    (label_width, label_height),
    (0, 0),
    (label_width, 0)
]

#layout set for 4 test labels
for i in range(4):
    if i < len(test_batch):
        item = test_batch.iloc[i]
        x, y = positions[i]

        #table - border
        pdf.rect(x, y, label_width, label_height)

        center_x = x + (label_width / 2)

        #IDENTIFIER
        pdf.setFont("Helvetica-Bold", 20)
        pdf.drawCentredString(center_x, y + 350, str(item['identifier']))

        #DESIGNER NAME
        pdf.setFont("Helvetica", 10)
        pdf.drawCentredString(center_x, y + 330, str(item['designer']))

        #IMAGE LOADER
        if pd.notna(item.get('Image_URL', '')):
            image_path = f"images/{item['Image_URL']}"
            if os.path.exists(image_path):
                try:
                    img = Image.open(image_path)
                    img.thumbnail((400, 400), Image.Resampling.LANCZOS)
                    img_buffer = BytesIO()
                    img.save(img_buffer, format='JPEG', quality=85)
                    img_buffer.seek(0)
                    img_reader = ImageReader(img_buffer)
                    pdf.drawImage(img_reader, center_x - 85, y + 110,
                                width=170, height=200, preserveAspectRatio=True)
                except:
                    pdf.drawCentredString(center_x, y + 200, "[No Image]")


        #DESCRIPTION
        pdf.setFont("Helvetica", 8)
        desc = str(item.get('description', ''))
        lines = textwrap.wrap(desc, width=70, max_lines=2, placeholder="...")
        if len(lines) >= 1:
          pdf.drawCentredString(center_x, y + 95, lines[0])
        if len(lines) >= 2:
          pdf.drawCentredString(center_x, y + 85, lines[1])

        #ITEM TYPE
        pdf.setFont("Helvetica-Bold", 11)
        pdf.drawCentredString(center_x, y + 60, str(item['item_type']))

        #INSTITUTION
        pdf.setFont("Helvetica-Bold", 9)
        pdf.drawCentredString(center_x, y + 30, "Pratt Fashion Design")
        pdf.drawCentredString(center_x, y + 15, "STUDY COLLECTION")

In [69]:
#Save our test label
pdf.save()
print("TEST_labels_with_desc.pdf is ready!")

TEST_labels_with_desc.pdf is ready!




---

#Chapter 3: Run it to the rest of dataset

Before running it out to the rest of the dataset. It would be a better pratice to have our filenaming system to navigate the pdf files easily. In many cases, we might include the date of when the file is developed with the insitution name. With this reasoning, we will use the [datetime](https://www.w3schools.com/python/python_datetime.asp) library from [W3S](https://www.w3schools.com/python/python_datetime.asp).

## Step 1: Setting up file naming system

In [70]:
#importing the datetime library
from datetime import datetime

- Putting the name of institution/collection

```ruby
today = datetime.now().strftime("%Y%m%d")
institution = "Pratt"  # ← CHANGE THIS TO YOUR INSTITUTION
```

- setting up your filename
```ruby
filename = f"{today}_{institution}_Garment_Bag_Labels.pdf"
counter = 1
```

- Setting up the counter, in case there are more than one cycle of dataset get processed in one day.
```ruby
while os.path.exists(filename):
    filename = f"{today}_{institution}_Garment_Bag_Labels_{counter}.pdf"
    counter += 1

```

##Step 2: Run the code for full dataset

In [56]:
#Filenaming system setup
today = datetime.now().strftime("%Y%m%d")
institution = "Pratt"  # ← CHANGE THIS TO YOUR INSTITUTION

#find available filename
filename = f"{today}_{institution}_Garment_Bag_Labels.pdf"
counter = 1

while os.path.exists(filename):
    filename = f"{today}_{institution}_Garment_Bag_Labels_{counter}.pdf"
    counter += 1

#Checking everything
print(f"📅 Date: {today}")
print(f"🏛️ Institution: {institution}")
print(f"📄 Creating: {filename}\n")

📅 Date: 20251129
🏛️ Institution: Pratt
📄 Creating: 20251129_Pratt_Garment_Bag_Labels.pdf



In [72]:
today = datetime.now().strftime("%Y%m%d")
institution = "Pratt"

filename = f"{today}_{institution}_Garment_Bag_Labels.pdf"
counter = 1

while os.path.exists(filename):
    filename = f"{today}_{institution}_Garment_Bag_Labels_{counter}.pdf"
    counter += 1

print(f"📄 Creating: {filename}\n")

# STEP 2: LOAD DATA
df = pd.read_csv('Garment_Bag_Tag_Dataset.csv')
total_items = len(df)
total_pages = (total_items + 3) // 4  #THIS LINE DEFINES total_pages

print(f"Processing {total_items} items on {total_pages} pages...\n")

# STEP 3: CREATE PDF
pdf = canvas.Canvas(filename, pagesize=letter)
width, height = letter
label_width = width / 2
label_height = height / 2

positions = [
    (0, label_height),
    (label_width, label_height),
    (0, 0),
    (label_width, 0)
]

# STEP 4: GENERATE ALL LABELS
for page_num in range(total_pages):
    start_idx = page_num * 4
    end_idx = min(start_idx + 4, total_items)
    batch = df.iloc[start_idx:end_idx]

    for i, (idx, item) in enumerate(batch.iterrows()):
        if i < 4:
            x, y = positions[i]

            # Border
            pdf.rect(x, y, label_width, label_height)
            center_x = x + (label_width / 2)

            # IDENTIFIER
            pdf.setFont("Helvetica-Bold", 20)
            pdf.drawCentredString(center_x, y + 350, str(item['identifier']))

            # DESIGNER
            pdf.setFont("Helvetica", 10)
            pdf.drawCentredString(center_x, y + 330, str(item['designer']))

            # IMAGE
            if pd.notna(item.get('Image_URL', '')):
                image_path = f"images/{item['Image_URL']}"
                if os.path.exists(image_path):
                    try:
                        img = Image.open(image_path)
                        img.thumbnail((400, 400), Image.Resampling.LANCZOS)
                        img_buffer = BytesIO()
                        img.save(img_buffer, format='JPEG', quality=85)
                        img_buffer.seek(0)
                        img_reader = ImageReader(img_buffer)
                        pdf.drawImage(img_reader, center_x - 85, y + 110,
                                    width=170, height=200, preserveAspectRatio=True)
                    except:
                        pdf.drawCentredString(center_x, y + 200, "[No Image]")

            # DESCRIPTION
            pdf.setFont("Helvetica", 8)
            desc = str(item.get('description', ''))
            lines = textwrap.wrap(desc, width=70, max_lines=2, placeholder="...")
            if len(lines) >= 1:
                pdf.drawCentredString(center_x, y + 95, lines[0])
            if len(lines) >= 2:
                pdf.drawCentredString(center_x, y + 85, lines[1])

            # ITEM TYPE
            pdf.setFont("Helvetica-Bold", 11)
            pdf.drawCentredString(center_x, y + 60, str(item['item_type']))

            # INSTITUTION
            pdf.setFont("Helvetica-Bold", 9)
            pdf.drawCentredString(center_x, y + 30, f"{institution} Fashion Design")
            pdf.drawCentredString(center_x, y + 15, "STUDY COLLECTION")

    if end_idx < total_items:
        pdf.showPage()

    if (page_num + 1) % 5 == 0:
        print(f"  ✓ Completed {page_num + 1} pages...")

        # STEP 5: SAVE AND DOWNLOAD
pdf.save()
print(f"\nSUCCESS! Created: {filename}")

📄 Creating: 20251129_Pratt_Garment_Bag_Labels.pdf

Processing 29 items on 8 pages...

  ✓ Completed 5 pages...

SUCCESS! Created: 20251129_Pratt_Garment_Bag_Labels.pdf


In [73]:
#check the pdf size
file_size = os.path.getsize(filename) / (1024 * 1024)
print(f"📦 File size: {file_size:.2f} MB")

📦 File size: 1.08 MB
